In [66]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import accuracy_score

# Create logistic regression model with batch gradient descent solver

## Set up sigmoid link function

$g(z) = \frac{1}{1 + exp(-z)}$

$h_\beta(x) = g(\beta^T x)$

Set all default weights to 1 (including bias term w).

In [6]:
def sigmoid(weights,features):
    z = np.dot(weights,features.T)
    hypo = 1.0/(1+np.exp(-z))
    return hypo

## Set up classifier function

In [7]:
def cl(hypo):
    predictions=hypo-(1-hypo)
    predictions[predictions>=0] = 1
    predictions[predictions<0] = 0
    return predictions

## Set up log-loss cost function

$J(\beta ) = -\frac{1}{m} \sum_{i=1}^m[y_i log(h_\beta (x_i) + (1 – y_i) log(1-h_\beta (x_i))]$

In [8]:
def logloss(targets,scores):
    m = len(targets)
    lloss = -(np.dot(targets,np.log(scores.T))+np.dot((1-targets),np.log(1-scores.T)))/m
    return lloss

## Set up gradient descent function

In [9]:
def grd(predictions,features,targets,alpha,weights):
    m = features.shape[1]
    grd=np.dot((predictions-targets),features)
    weights_updated=weights-np.dot(alpha,grd)
    return weights_updated

## Build logistic regression function

In [173]:
def logistic_regression(features,targets, tol=0.0001, max_iter=100):
    # reshape targets
    targets = targets.reshape(1,-1)
    
    # augment feature vector to accomdate bias term in the model
    features = np.hstack((features,np.ones((features.shape[0],1))))
    
    # initialize weights
    weights = np.matrix(np.ones((1,features.shape[1])))
    
    # compute initial scores (hypothesis out)
    scores = sigmoid(weights,features)
    
    # compute initial predictions
    predictions = cl(scores)
    
    # compute initial logloss
    cost = logloss(targets,scores)
    
    # use batch gradient descent to estimate model coefficients
    n_iterations = max_iter
    learning_rate = 0.001 # set default learning rate to 0.001
    cost_delta = tol
    
    while n_iterations>0 and cost_delta>=tol: # set up stopping criteria
        weights_updated = grd(predictions,features,targets,learning_rate,weights)
        scores = sigmoid(weights_updated,features)
        predictions = cl(scores)
        cost_updated = logloss(targets,scores)
        cost_delta = cost - cost_updated
        
        cost = cost_updated
        n_iterations-=1
        
        if cost_delta > 0:
            weights = weights_updated
        else:
            pass
        
    return weights,predictions

# Test model and solver on Titanic dataset

## Load preprocessed training data

In [203]:
train_X = pd.read_csv('../Data/train_X.csv')
train_y = pd.read_csv('../Data/train_y.csv',squeeze=True)
test_X = pd.read_csv('../Data/test_X.csv')
test_y = pd.read_csv('../Data/test_y.csv',squeeze=True)

In [204]:
train_X = np.matrix(train_X)
train_y = np.array(train_y)
test_X = np.matrix(test_X)
test_y = np.array(test_y)

## Fit hand-made LR model on data

In [174]:
w,predictions = logistic_regression(train_X,train_y)

In [175]:
w

matrix([[ 0.07579007, -0.09043556, -0.13758667, -0.04715164,  0.49496346,
         -0.273     ,  0.541     ,  0.247     , -0.01      ]])

### Check training accuracy

In [176]:
accuracy_score(train_y, predictions.reshape(-1,1))

0.7223113964686998

### Check test accuracy

In [202]:
test_X_aug = np.hstack((test_X,np.ones((test_X.shape[0],1))))

In [188]:
test_scores = sigmoid(w,test_X_aug)

In [196]:
test_predictions = cl(test_scores).T

In [197]:
accuracy_score(test_predictions,test_y)

0.73134328358208955

## Fit sklearn LR model on data 

In [68]:
lr_sklearn = lr(C=1e15)

In [71]:
lr_sklearn.fit(train_X,train_y)

LogisticRegression(C=1e+15, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [73]:
lr_sklearn.coef_

array([[-0.853682  , -0.57435996, -0.34506289, -0.03165663,  0.10383695,
        -2.82151961, -0.19328792, -0.32573504]])

In [74]:
lr_sklearn.intercept_

array([ 1.41905563])

### Check training accuracy

In [129]:
lr_sklearn.score(train_X,train_y)

0.8025682182985554

### Check test accuracy

In [205]:
lr_sklearn.score(test_X,test_y)

0.79104477611940294